In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
import warnings
import time
import os
from collections import defaultdict

# Suppress warnings for cleaner output
warnings.filterwarnings("ignore", category=FutureWarning)

def load_and_prepare_data(file_path):
    """
    Loads basket data, sets efficient data types, and creates mappings.
    """
    print("Step 1: Loading and preparing data...")
    if not os.path.exists(file_path):
        print(f"Error: File not found at {file_path}")
        return None, None, None
    
    df_baskets = pd.read_csv(file_path)
    df_baskets['order_id'] = df_baskets['order_id'].astype('category')
    df_baskets['product_id'] = df_baskets['product_id'].astype('category')
    
    product_ids = df_baskets['product_id'].cat.categories
    product_to_index = {product: i for i, product in enumerate(product_ids)}
    index_to_product = {i: product for product, i in product_to_index.items()}
    
    print("Data loaded and prepared successfully.")
    return df_baskets, product_to_index, index_to_product

def build_item_order_matrix(df_baskets):
    """
    Builds a sparse item-order matrix (X) where X[i, j] = 1 if item i is in order j.
    """
    print("\nStep 2: Building the item-order sparse matrix...")
    n_products = len(df_baskets['product_id'].cat.categories)
    n_orders = len(df_baskets['order_id'].cat.categories)
    
    product_codes = df_baskets['product_id'].cat.codes.values
    order_codes = df_baskets['order_id'].cat.codes.values
    data = np.ones(len(df_baskets))
    
    item_order_matrix = csr_matrix((data, (product_codes, order_codes)), shape=(n_products, n_orders))
    
    print("Item-order matrix built successfully.")
    return item_order_matrix

def compute_topk_neighbors(item_order_matrix, index_to_product, K=10):
    """
    Computes Top-K neighbors for each item using the efficient row-wise method.
    """
    print("\nStep 3 & 4: Computing Top-K neighbors row-by-row...")
    start_time = time.time()
    n_products = item_order_matrix.shape[0]
    top_k_similar_items = {}
    freq = np.asarray(item_order_matrix.sum(axis=1)).ravel()
    sqrt_freq = np.sqrt(np.clip(freq, 1e-12, None))

    for i in range(n_products):
        product_id = index_to_product[i]
        co_row = item_order_matrix.getrow(i).dot(item_order_matrix.T)
        neighbor_indices = co_row.indices
        co_occurrence_values = co_row.data
        denominator = sqrt_freq[i] * sqrt_freq[neighbor_indices]
        similarity_scores = co_occurrence_values / np.clip(denominator, 1e-12, None)
        
        mask = neighbor_indices != i
        neighbor_indices = neighbor_indices[mask]
        similarity_scores = similarity_scores[mask]
        
        if len(similarity_scores) == 0:
            top_k_similar_items[product_id] = []
            continue

        k_to_find = min(K, len(similarity_scores))
        top_k_partition_indices = np.argpartition(similarity_scores, -k_to_find)[-k_to_find:]
        
        final_indices = neighbor_indices[top_k_partition_indices]
        final_scores = similarity_scores[top_k_partition_indices]
        sort_order = np.argsort(final_scores)[::-1]
        
        top_k_similar_items[product_id] = [
            {'product_id': index_to_product[idx], 'score': score}
            for idx, score in zip(final_indices[sort_order], final_scores[sort_order])
        ]
        
    print(f"Top-K calculation done in {time.time() - start_time:.2f} seconds.")
    return top_k_similar_items

def save_neighbors_to_file(top_k_items, file_path):
    """
    Saves the computed neighbors to a CSV file for easy reuse.
    """
    print(f"\nSaving neighbors artifact to {file_path}...")
    rows = []
    for product_id, neighbors in top_k_items.items():
        for neighbor in neighbors:
            rows.append([product_id, neighbor['product_id'], neighbor['score']])
    
    df_neighbors = pd.DataFrame(rows, columns=['product_id', 'neighbor_id', 'score'])
    df_neighbors.to_csv(file_path, index=False)
    print("Artifact saved successfully.")

def get_candidate_items(user_recent_purchases, top_k_similar_items, num_recommendations=5):
    """
    Generates candidate items for a user based on their recent purchases.
    """
    candidate_items = defaultdict(float)
    recent_set = set(user_recent_purchases)

    for product_id in user_recent_purchases:
        if product_id in top_k_similar_items:
            for item in top_k_similar_items[product_id]:
                candidate_product = item['product_id']
                score = item['score']

                if candidate_product in recent_set:
                    continue
                
                if score > candidate_items[candidate_product]:
                    candidate_items[candidate_product] = score

    sorted_candidates = sorted(candidate_items.items(), key=lambda x: x[1], reverse=True)
    return [item[0] for item in sorted_candidates[:num_recommendations]]

In [ ]:
def main():
    """
    Main function to run the entire pipeline.
    """
    # Define file paths
    # IMPORTANT: Update this path to your actual file location
    DATA_FILE_PATH = r"C:\Users\kthac\Desktop\Group_4419\item item similarity\item_item_basket.csv"
    NEIGHBORS_ARTIFACT_PATH = "top_k_neighbors.csv"
    
    # Load and prepare data
    df_baskets, product_to_index, index_to_product = load_and_prepare_data(DATA_FILE_PATH)
    
    if df_baskets is None:
        return # Exit if data loading failed

    # Build the item-order matrix
    item_order_matrix = build_item_order_matrix(df_baskets)
    
    # Compute the neighbors
    top_k_items = compute_topk_neighbors(item_order_matrix, index_to_product, K=10)
    
    # Save the results for future use
    save_neighbors_to_file(top_k_items, NEIGHBORS_ARTIFACT_PATH)
    
    # --- Example Usage ---
    print("\nStep 5: Generating candidate items for a user...")
    
    # Sample user purchases (using popular product IDs)
    user_purchases = [13176, 21137]
    recommended_candidates = get_candidate_items(user_purchases, top_k_items, num_recommendations=5)
    
    print(f"\nCandidate items for a user who recently bought products {user_purchases}:")
    print(recommended_candidates)
    
    # Show sample output for a specific product
    print("\nTop 3 Similar Items for a sample product (ID: 24852):")
    if 24852 in top_k_items and top_k_items[24852]:
        top_3 = top_k_items[24852][:3]
        print(top_3)
    else:
        print("Sample product ID 24852 not found or has no similar items.")
        
    print("\nProcess finished successfully.")

# Run the main function
if __name__ == '__main__':
    main()

Step 1: Loading and preparing data...
Data loaded and prepared successfully.

Step 2: Building the item-order sparse matrix...
Item-order matrix built successfully.

Step 3 & 4: Computing Top-K neighbors row-by-row...


# Step 1: Data Loading and Initial Preparation
This section handles data and initial memory optimization using pandas.
pd.read_csv: Loads the basket and product count data into pandas DataFrames.
.astype('category'): This is a key memory-saving step. Instead of storing product and order IDs as int64 or object strings, they are converted to a category dtype. Pandas internally maps each unique ID to a contiguous integer (accessible via .cat.codes), which is much more memory-efficient and provides a direct lookup for creating our sparse matrix.
ID to Index Mapping: Dictionaries (product_to_index, index_to_product) are created to map the original product_id to its new integer index (0 to n_products-1) and back. This is necessary because the matrix will operate on these zero-based indices.

# Step 2: Creating the Item-Order Sparse Matrix
The goal here is to create a binary matrix representing which items appeared in which orders. A (num_products x num_orders) NumPy array was too large.
scipy.sparse.csr_matrix: We use a Compressed Sparse Row (CSR) matrix because it's highly efficient for both storage and, more importantly, for the fast matrix-vector and matrix-matrix multiplications (.dot() product) required in the next step.
csr_matrix((data, (rows, cols)), shape=...): This is the standard constructor for creating a sparse matrix from coordinate-style data.
rows: The product indices (df_baskets['product_id'].cat.codes).
cols: The order indices (df_baskets['order_id'].cat.codes).
data: A list of ones, indicating the presence of an item in an order.

# Step 3: Calculate Item-Item Similarity 
This is the core of the algorithm. It computes a similarity score between every pair of products.
Co-occurrence Matrix: The line co_occurrence_matrix = item_order_matrix.dot(item_order_matrix.T) is the most critical calculation. By taking the dot product of the matrix with its transpose, we efficiently compute the co-occurrence matrix. The resulting (num_products x num_products) sparse matrix contains the raw count of how many times any two items appeared together in the same basket.
Memory Management: A dense float matrix of the full similarity scores would be too large to fit in memory (~18 GB for 49k products). The code therefore processes the similarity calculation in chunks.
Similarity Metric: The metric used is a normalized co-occurrence. The formula is essentially Co-occurrence(A, B) / (sqrt(Count(A)) * sqrt(Count(B))).
np.outer: The denominator is calculated efficiently for each chunk using NumPy's outer product. This broadcasts the vectors of square-rooted product frequencies, avoiding a slow, explicit Python loop.
lil_matrix: The results are stored in a List of Lists (LIL) matrix. lil_matrix is chosen here because it's efficient for constructing sparse matrices incrementally by changing sparsity structure (i.e., filling in the chunks row by row).

# Step 4: Find Top-K Similar Items
This section extracts the top K most similar items for each product from the final similarity matrix.
.tocsr(): The similarity matrix is converted from LIL back to CSR format. CSR is much more efficient for row slicing, which is exactly what we're about to do in the loop.
np.argsort(): Instead of using a computationally expensive method like sklearn.NearestNeighbors (which would require a dense distance matrix), the code uses a more direct and faster NumPy approach.
np.argsort(row) returns the indices that would sort the array in ascending order.
By reversing this with [::-1], we get the indices of the similarity scores from largest to smallest.
Slicing with [:K+1] gives the indices of the top K+1 items (the item itself plus its K nearest neighbors). This is an extremely fast and memory-efficient way to perform a top-k search on each row.

# Step 5: Generate Outputs (Example Usage)
This final part is a practical demonstration of how to use the pre-computed similarities to generate recommendations.
get_candidate_items function: This function takes a list of a user's recent purchases and returns a ranked list of recommended products.
Candidate Aggregation: It iterates through the user's items and their corresponding similarity lists. A dictionary (candidate_items) is used to aggregate all potential recommendations.
Score Deduplication: Using a dictionary is a clever way to handle duplicates. The line if candidate_product not in candidate_items or score > candidate_items[candidate_product] ensures that if a candidate is suggested by multiple user items, only the highest similarity score is retained.
Final Ranking: Finally, sorted(candidate_items.items(), ...) is used with a lambda function to sort the candidates by their similarity scores in descending order before returning the top N product IDs.